In [1]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Data processing**

In [3]:
#Training data

import csv

# Initialize an empty dictionary
train_data_dict = {}
train_data_dict["Scores"]=[]
train_data_dict["Sentence_1"]=[]
train_data_dict["Sentence_2"]=[]

# Open the CSV file
with open('train.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    reader=list(reader)
    # Iterate over each row in the CSV file
    for i in range(1,len(reader)):
        row=reader[i]
        if len(row)==1:
            row=row[0].split('\t')
        else:
            row=''.join(row).split('\t')
        #print(row)
        train_data_dict["Scores"].append(row[0])
        train_data_dict["Sentence_1"].append(row[1])
        train_data_dict["Sentence_2"].append(row[2])




In [4]:
#Now get data embeddings
train_data_sentence_1=model.encode(train_data_dict["Sentence_1"])
train_data_sentence_2=model.encode(train_data_dict["Sentence_2"])

In [5]:
#Checking error cases

problem_indices=[]
for i in range(len(train_data_dict["Scores"])):
    try:
        train_data_dict["Scores"][i]=float(train_data_dict["Scores"][i])
    except ValueError:
        #print("Error at i =",i,val_data_dict["Scores"][i])
        problem_indices.append(i)

In [6]:
len(problem_indices)

38

In [7]:
#Dropping the error cases
X_train_s1=[train_data_sentence_1[i] for i in range(len(train_data_sentence_1)) if i not in problem_indices]
X_train_s2=[train_data_sentence_2[i] for i in range(len(train_data_sentence_2)) if i not in problem_indices]
Y_train=[train_data_dict["Scores"][i] for i in range(len(train_data_dict["Scores"])) if i not in problem_indices]


In [8]:
#Data for 1A and 1C(remove faulty sentences)
X_train_sentence1=[train_data_dict["Sentence_1"][i] for i in range(len(train_data_dict["Sentence_1"])) if i not in problem_indices]
X_train_sentence2=[train_data_dict["Sentence_2"][i] for i in range(len(train_data_dict["Sentence_2"])) if i not in problem_indices]

In [9]:
len(X_train_sentence1)

5711

### Validation data

In [10]:
val_data_dict = {}
val_data_dict["Scores"]=[]
val_data_dict["Sentence_1"]=[]
val_data_dict["Sentence_2"]=[]

# Open the CSV file
with open('dev.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    reader=list(reader)
    # Iterate over each row in the CSV file
    for i in range(1,len(reader)):
        row=reader[i]
        if len(row)==1:
            row=row[0].split('\t')
        else:
            row=''.join(row).split('\t')
        #print(row)
        val_data_dict["Scores"].append(row[0])
        val_data_dict["Sentence_1"].append(row[1])
        val_data_dict["Sentence_2"].append(row[2])



In [11]:
#Now get data embeddings
val_data_sentence_1=model.encode(val_data_dict["Sentence_1"])
val_data_sentence_2=model.encode(val_data_dict["Sentence_2"])

In [12]:
#Saving the embeddings for future use
Val_data_embeddings={}
Val_data_embeddings["Sentence_1"]=val_data_sentence_1
Val_data_embeddings["Sentence_2"]=val_data_sentence_2

import pickle as pkl
with open('SBERTValDataEmbeddings.pkl','wb') as file:
    pkl.dump(Val_data_embeddings,file)

In [13]:
type(val_data_sentence_1)

numpy.ndarray

In [14]:
len(val_data_sentence_1)

1474

In [15]:
val_data_sentence_1.shape

(1474, 384)

In [16]:
val_data_sentence_2.shape

(1474, 384)

In [17]:
#Checking error cases

problem_indices=[]
for i in range(len(val_data_dict["Scores"])):
    try:
        val_data_dict["Scores"][i]=float(val_data_dict["Scores"][i])
    except ValueError:
        #print("Error at i =",i,val_data_dict["Scores"][i])
        problem_indices.append(i)

In [18]:
len(problem_indices)

30

In [19]:
#Drop the rows that give value error
X_val_s1=[val_data_sentence_1[i] for i in range(len(val_data_sentence_1)) if i not in problem_indices]
X_val_s2=[val_data_sentence_2[i] for i in range(len(val_data_sentence_2)) if i not in problem_indices]
Y_val=[val_data_dict["Scores"][i] for i in range(len(val_data_dict['Scores'])) if i not in problem_indices]


In [20]:
#Data for 1A and 1C(remove faulty sentences)
X_val_sentence1=[val_data_dict["Sentence_1"][i] for i in range(len(val_data_dict["Sentence_1"])) if i not in problem_indices]
X_val_sentence2=[val_data_dict["Sentence_2"][i] for i in range(len(val_data_dict["Sentence_2"])) if i not in problem_indices]

In [ ]:
len(X_val_sentence1)

1444

# **Setup 1B**

**Cosine Similarity**

In [21]:
# import required libraries
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
# # define two lists or array
# A = np.array([2,1,2,3,2,9])
# B = np.array([3,4,2,4,5,5])

# print("A:", A)
# print("B:", B)

# compute cosine similarity for val data

cosine_similarity_scores=[]
for i in range(len(X_val_s1)):
    A=np.array(X_val_s1[i]).reshape(1,-1)
    B=np.array(X_val_s2[i]).reshape(1,-1)
    cosine=cosine_similarity(A,B)
    # cosine = np.dot(A,B)/(norm(A)*norm(B))
    cosine_similarity_scores.append(cosine)

#print("Cosine Similarity:", cosine)


In [23]:
type(cosine_similarity_scores[0][0])

IndexError: invalid index to scalar variable.

In [22]:
cosine_similarity_scores=[cosine_similarity_scores[i][0][0] for i in range(len(cosine_similarity_scores))]

In [25]:
cosine_similarity_scores

[0.99337167,
 0.95396215,
 0.98010147,
 0.64708304,
 0.7462275,
 0.7546188,
 0.98153895,
 0.6809811,
 0.891471,
 0.95492184,
 0.86552733,
 0.5417836,
 0.98097706,
 0.9263348,
 0.9664142,
 0.061895043,
 0.79292023,
 0.9568715,
 0.9118954,
 0.36104676,
 0.72878253,
 0.5095633,
 0.7969234,
 0.5152466,
 0.38104507,
 0.5949642,
 0.7490593,
 0.6846779,
 0.34796557,
 -0.039455414,
 0.7508143,
 0.8956351,
 0.90359855,
 0.97901076,
 0.8268659,
 0.9053811,
 0.9832699,
 0.97837657,
 0.90206987,
 0.42854583,
 0.78560644,
 0.7909082,
 0.33931527,
 0.7585868,
 0.53844494,
 0.25695264,
 -0.095880896,
 -0.057387955,
 0.77093506,
 0.8934878,
 0.33681694,
 0.8569606,
 0.8171691,
 0.9618431,
 0.24741846,
 0.39973292,
 0.19585983,
 0.14628725,
 0.7464845,
 -0.044956297,
 0.74602413,
 0.8157452,
 0.5057204,
 0.8486227,
 0.7478349,
 0.035063006,
 0.86091006,
 0.8122051,
 0.6200373,
 0.8680935,
 0.7488051,
 0.54953766,
 0.6661395,
 0.13215528,
 0.16289924,
 0.90115476,
 0.6778492,
 0.62001073,
 0.37883902,
 

In [26]:
max(cosine_similarity_scores)

1.0000001

In [27]:
cos_2=[round(i,3) for i in cosine_similarity_scores]

In [28]:
max(cos_2)

1.0

In [29]:
min(cos_2)

-0.115

In [30]:
len(cosine_similarity_scores)

1444

In [31]:
min_val=abs(min(cos_2))

In [ ]:
min_val

0.115

In [32]:
#scale the cosine similaity scores\
# scaled_cosine=[]
# for i in cos_2:
#     if i<0:
#         scaled_cosine.append(5*(i+1))
#     else:
#         scaled_cosine.append(5*i)
f=5/1.115
scaled_cosine=[round((i+min_val)*f,3) for i in cos_2]

In [33]:
max(scaled_cosine)

5.0

In [34]:
min(scaled_cosine)

0.0

In [35]:
type(scaled_cosine[0])

numpy.float64

In [36]:
type(val_data_dict["Scores"][0])

float

In [37]:
min(Y_val)

0.0

In [38]:
max(scaled_cosine)

5.0

In [39]:
# Pearson Correlation(CHECK THIS)
from scipy.stats import pearsonr


# Apply the pearsonr()
corr, _ = pearsonr(np.array(scaled_cosine), np.array(Y_val))
print('Pearsons correlation: %.3f' % corr)


Pearsons correlation: 0.860
